In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim, mean_squared_error


img_path = 'sar_1.jpg'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  


def add_gaussian_noise(image, mean=0, sigma=25):
    noise = np.random.normal(mean, sigma, image.shape).astype(np.int16)  
    noisy = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return noisy

def add_constant_noise(image, a=110, b=0):
    mid = (a + b) / 2
    stdev = a - mid
    noise = np.random.normal(mid, stdev, image.shape).astype(np.int16)
    noisy = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return noisy

img_gn = add_gaussian_noise(img)
img_const = add_constant_noise(img)


def apply_cv2_filters(noisy_images):
    filters = {
        'Median 3x3': {
            'gn': cv2.medianBlur(noisy_images['gn'], 3),
            'constant': cv2.medianBlur(noisy_images['constant'], 3)
        },
        'Median 5x5': {
            'gn': cv2.medianBlur(noisy_images['gn'], 5),
            'constant': cv2.medianBlur(noisy_images['constant'], 5)
        },
        'Gaussian 3x3': {
            'gn': cv2.GaussianBlur(noisy_images['gn'], (3,3), 0),
            'constant': cv2.GaussianBlur(noisy_images['constant'], (3,3), 0)
        },
        'Gaussian 5x5': {
            'gn': cv2.GaussianBlur(noisy_images['gn'], (5,5), 0),
            'constant': cv2.GaussianBlur(noisy_images['constant'], (5,5), 0)
        },
        'Bilateral d=9': {
            'gn': cv2.bilateralFilter(noisy_images['gn'], 9, 75, 75),
            'constant': cv2.bilateralFilter(noisy_images['constant'], 9, 75, 75)
        },
        'Bilateral d=15': {
            'gn': cv2.bilateralFilter(noisy_images['gn'], 15, 75, 75),
            'constant': cv2.bilateralFilter(noisy_images['constant'], 15, 75, 75)
        },
        'NL Means h=10': {
            'gn': cv2.fastNlMeansDenoising(noisy_images['gn'], None, 10, 7, 21),
            'constant': cv2.fastNlMeansDenoising(noisy_images['constant'], None, 10, 7, 21)
        },
        'NL Means h=20': {
            'gn': cv2.fastNlMeansDenoising(noisy_images['gn'], None, 20, 7, 21),
            'constant': cv2.fastNlMeansDenoising(noisy_images['constant'], None, 20, 7, 21)
        }
    }
    return filters

noisy_images = {'gn': img_gn, 'constant': img_const}
filters = apply_cv2_filters(noisy_images)


def calculate_psnr(img1, img2):
    mse = np.mean((img1.astype(np.float32) - img2.astype(np.float32)) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

def calculate_metrics(original, filtered):
    mse_val = mean_squared_error(original, filtered)
    ssim_val = ssim(original, filtered)
    psnr_val = calculate_psnr(original, filtered)
    return mse_val, ssim_val, psnr_val


def print_results(results):
    print("Результаты фильтрации (MSE, SSIM, PSNR):")
    print("\nШум Гаусса:")
    print("Фильтр\t\t\tMSE\t\tSSIM\t\tPSNR")
    for filter_name, metrics in results.items():
        mse, ssim_val, psnr_val = metrics['Gaussian']
        print(f"{filter_name:20}\t{mse:.2f}\t\t{ssim_val:.4f}\t\t{psnr_val:.2f}")

    print("\nПостоянный шум:")
    print("Фильтр\t\t\tMSE\t\tSSIM\t\tPSNR")
    for filter_name, metrics in results.items():
        mse, ssim_val, psnr_val = metrics['Constant']
        print(f"{filter_name:20}\t{mse:.2f}\t\t{ssim_val:.4f}\t\t{psnr_val:.2f}")


def evaluate_filters(original, filters):
    results = {}
    for filter_name, filtered_imgs in filters.items():
        results[filter_name] = {
            'Gaussian': calculate_metrics(original, filtered_imgs['gn']),
            'Constant': calculate_metrics(original, filtered_imgs['constant'])
        }
    return results

results = evaluate_filters(img, filters)
print_results(results)


def plot_original_and_noisy(original, noisy_gn, noisy_constant):
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.imshow(original, cmap='gray')
    plt.title("Оригинальное изображение")
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(noisy_gn, cmap='gray')
    plt.title("Шум Гаусса")
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(noisy_constant, cmap='gray')
    plt.title("Постоянный шум")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def plot_filtered_results(filters, noise_type, title):
    plt.figure(figsize=(20,10))
    plt.suptitle(title, fontsize=16)
    for i, (filter_name, filtered_imgs) in enumerate(filters.items()):
        plt.subplot(2, 4, i+1)
        plt.imshow(filtered_imgs[noise_type], cmap='gray')
        plt.title(filter_name)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

plot_original_and_noisy(img, img_gn, img_const)
plot_filtered_results(filters, 'gn', "Фильтры для шума Гаусса")
plot_filtered_results(filters, 'constant', "Фильтры для постоянного шума")


ModuleNotFoundError: No module named 'cv2'